In [4]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt
import scipy
# from scipy import linalg
# from scipy import ndimage
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import pickle


In [5]:
# DEKLARASI PATH DATA TRAIN DAN TEST
TrainingImagePath=  r"..\visila\skin-disease-datasaet\train_set"
TestImagePath = r"..\visila\skin-disease-datasaet\test_set"

# IMAGE DATA GENERATOR -> UNTUK AUGMENTASI GAMBAR PADA DATASHEET TRAINING
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1/255,
        rotation_range=10,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        zoom_range=0.05,
        horizontal_flip=True,
        fill_mode='nearest')

# MEMBUAT OBJEK UNTUK RESCALLING PADA DATASHEET TRAINING
test_datagen = ImageDataGenerator(rescale=1/255)

# MEMBUAT DATASHEET TRAINING DARI DIREKTORI, GAMBAR AKAN DIRUBAH SESUAI DENGAN KONFIGURASI YANG DITENTUKAN
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')  # KARENA DATA INI BERKATEGORI BUKAN BINARY

# MEMBUAT DATASHEET TESTING DARI DIREKTORI, GAMBAR AKAN DIRUBAH SESUAI DENGAN KONFIGURASI YANG DITENTUKAN
test_set = test_datagen.flow_from_directory(
        TestImagePath,
        target_size=(64, 64),
        batch_size=16,
        class_mode='categorical')

# MENDAPATKAN INDEKS CLASS YANG SUDAH DITEMUKAN DALAM DATASHEET PENGUJIAN
test_set.class_indices

Found 924 images belonging to 8 classes.
Found 233 images belonging to 8 classes.


{'BA- cellulitis': 0,
 'BA-impetigo': 1,
 'FU-athlete-foot': 2,
 'FU-nail-fungus': 3,
 'FU-ringworm': 4,
 'PA-cutaneous-larva-migrans': 5,
 'VI-chickenpox': 6,
 'VI-shingles': 7}

In [6]:
# MENGAMBIL INDEKS DARI DATASHEET CLASS SEBELUMNYA
TrainClasses = training_set.class_indices

# MEMBUAT PEMETAAN YANG HASILNYA DALAM BENTUK DICTIONARY
ResultMap = {}
for faceValue, faceName in zip(TrainClasses.values(), TrainClasses.keys()):
    ResultMap[faceValue] = faceName

with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)

# MENAMPILKAN PEMETAAN ID CLASS DAN NAMA CLASS
print("Mapping of Face and its ID", ResultMap)

Mapping of Face and its ID {0: 'BA- cellulitis', 1: 'BA-impetigo', 2: 'FU-athlete-foot', 3: 'FU-nail-fungus', 4: 'FU-ringworm', 5: 'PA-cutaneous-larva-migrans', 6: 'VI-chickenpox', 7: 'VI-shingles'}


In [7]:
OutputNeurons = len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

# MEMBUAT MODEL
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf

# MENDEFINISIKAN FUNSGI UNTUK MENJADI MODEL
def create_model():
    # MODEL SEKUENSIAL
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64, 64, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(OutputNeurons, activation='softmax')
    ])
    # model = tf.keras.models.Sequential([
    #     tf.keras.layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64, 64, 3), activation='relu'),
    #     tf.keras.layers.MaxPooling2D(2, 2),
    #     tf.keras.layers.Conv2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'),
    #     tf.keras.layers.MaxPooling2D(2, 2),
    #     tf.keras.layers.Flatten(),
    #     tf.keras.layers.Dense(64, activation='relu'),
    #     tf.keras.layers.Dense(OutputNeurons, activation='softmax')
    # ])

    # MENGKOMPILASI PROGRAM
    model.compile(optimizer='adam',  # MENGGUNAKAN ADAM KARENA DATASHEET BERKATEGORI DAN YANG MENDUKUNG ADALAH ADAM
                  loss='categorical_crossentropy',  # DATA BERKATEGORI BUKAN BINARY
                  metrics=['accuracy'])  # YANG INGIN DILIHAT NILAI AKURASINYA

    # MENAMPILKAN RINGKASAN MODEL
    model.summary()
    return model

# PANGGIL FUNGSI MODEL
model = create_model()


 The Number of output neurons:  8



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 60, 32)        2432      
                                                                 
 max_pooling2d (MaxPooling2  (None, 30, 30, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 10816)             0         
                                                                 
 dense (Dense)    

In [10]:
history = model.fit(training_set,
                    epochs=50,
                    validation_data=test_set
                    # validation_steps=10
                    )


Epoch 1/50


29/29 [==============================] - 4s 95ms/step - loss: 2.0337 - accuracy: 0.1905 - val_loss: 1.8827 - val_accuracy: 0.3004
Epoch 2/50
29/29 [==============================] - 2s 84ms/step - loss: 1.8190 - accuracy: 0.3149 - val_loss: 1.8521 - val_accuracy: 0.2704
Epoch 3/50
29/29 [==============================] - 2s 79ms/step - loss: 1.6199 - accuracy: 0.4069 - val_loss: 1.6645 - val_accuracy: 0.3991
Epoch 4/50
29/29 [==============================] - 2s 81ms/step - loss: 1.4915 - accuracy: 0.4600 - val_loss: 1.5514 - val_accuracy: 0.4721
Epoch 5/50
29/29 [==============================] - 2s 77ms/step - loss: 1.3281 - accuracy: 0.5368 - val_loss: 1.4925 - val_accuracy: 0.4506
Epoch 6/50
29/29 [==============================] - 2s 77ms/step - loss: 1.2818 - accuracy: 0.5390 - val_loss: 1.4962 - val_accuracy: 0.4807
Epoch 7/50
29/29 [==============================] - 2s 77ms/step - loss: 1.2160 - accuracy: 0.5628 - val_loss: 1.3299 - val_accuracy: 0.5021
Epoch 8/50


Image 

In [43]:
# Load the ResultMap from the pickle file
with open("ResultsMap.pkl", 'rb') as fileReadStream:
    ResultMap = pickle.load(fileReadStream)


image_path = "../visila/skin-disease-datasaet/train_set/FU-ringworm/0_FU-ringworm (38).jpg"

frame = cv2.imread(image_path)

print("Original Image Shape:", frame.shape)


# Resize the image to match the input size expected by your model
resized_frame = cv2.resize(frame, (64, 64))

# Normalize the image
img_array = np.expand_dims(resized_frame, axis=0) / 255.0

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class label
predicted_class = ResultMap[np.argmax(predictions)]

# Display the prediction on the image
cv2.putText(frame, f'Prediction: {predicted_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image with the prediction
cv2.imshow('Image Prediction', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

Original Image Shape: (224, 224, 3)
1/1 [==============================] - 0s 22ms/step


Video

In [17]:
with open("ResultsMap.pkl", 'rb') as fileReadStream:
    ResultMap = pickle.load(fileReadStream)

cap = cv2.VideoCapture(0)  

while True:
    ret, frame = cap.read()
    resized_frame = cv2.resize(frame, (64, 64))
    img_array = np.expand_dims(resized_frame, axis=0) / 255.0
    predictions = model.predict(img_array)
    predicted_class = ResultMap[np.argmax(predictions)]
    cv2.putText(frame, f'Prediction: {predicted_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Webcam Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 19ms/step


KeyboardInterrupt: 